В данном задании вам нужно будет провести анализ данных проведенного A/B-теста.

Данные будут предоставлены в формате csv. Они будут содержать также часть пред экспериментального этапа.

### Описание эксперимента:
Есть мобильное приложение. В этом приложении у пользователей есть возможность покупать игровые предметы за реальные деньги. Чтобы стимулировать пользователей их покупать, приложение периодически предлагает пользователям товары - появляется окошко с рекомендацией купить товар. Отдел машинного обучения предложил улучшение для текущего алгоритма выбора рекомендации. Для проверки улучшений алгоритма был проведен A/B тест. Лог его проведения предоставлен в прикрепленном файле. 

### Метрика: средний доход от пользователя за 1 неделю после первого показа ему рекомендации на 10% (после начала A/B теста время первого показа ищется снова)

Важная информация:
Эксперимент начинается 2023-05-01. Данные есть до 2023-06-01 (но можно завершить раньше, если это позволит оценка длительности)
Вам сказали, что его длительность должна составить 1 месяц.
Все покупки, которые вызваны не влиянием рекомендаций, в этом логе не учитываются

### Описание данных:
id_product -  идентификатор продукта, который был рекомендован
is_pay - купил ли пользователь товар
sum_payment - размер платежа (0, если не купил)
city - город, в котором находится пользователь
id_user - пользователь
timestamp - timestamp события
date - дата события

### Задачи, которые необходимо решить:
1) Оценить длительность теста на момент его начала. Сравнить с предложенной. Для оценки необходимо использовать данные с пред экспериментального периода. 
2) Посмотреть, есть ли выбросы в данных.
3) Построить методику расчета целевой метрики. 
4) Рассчитать целевую метрику на день окончания теста (рассчитанной в п1) для группы A и B, рассчитать эффект, p_value. 
5) Посмотреть, есть ли выбросы в данных.
6) Рассчитать метрики из п2 по дням и построить их графики.
7) Принять решение о результате теста - обосновать.

### Дополнительно:
При подсчете метрик обработать выбросы в данных - описать почему их можно удалить из теста, если они удаляются. Пожалуйста, не заглядывайте в будущее по данным, если рассчитанная длительность меньше предоставленных данных.

Данные: ссылка, читать стоит с pd.read_csv("ab_made_4.gzip", compression='gzip')

In [529]:
import pandas as pd
import numpy as np
import random
import scipy.stats as sps
import plotly.graph_objs as go

In [530]:
path = 'data/ab_made_4.csv'
df = pd.read_csv(path)

In [531]:
df.head(5)

,timestamp,id_user,sum_payment,group,city,id_product,is_pay,date
0,1680330573,user_9903,27,NaN,Санкт-Петербург,4.0,1,2023-04-01
1,1680332652,user_6732,0,NaN,Рязань,1.0,0,2023-04-01
2,1680378039,user_4199,0,NaN,Москва,3.0,0,2023-04-01
3,1680337580,user_3606,12,NaN,Санкт-Петербург,7.0,1,2023-04-01
4,1680334389,user_9519,0,NaN,Санкт-Петербург,14.0,0,2023-04-01


In [532]:
df.shape

(56405, 8)

### Оценить длительность теста на момент его начала. Сравнить с предложенной. Для оценки необходимо использовать данные с пред экспериментального периода. Посмотреть, есть ли выбросы в данных

Нам даны исторические данные за предыдущий месяц до запуска эсперимента.
Посмотрим, как выглядят покупки клиентов за этот период, есть ли выбросы.
Так как нам нужно оценить длительность эксперимента исходя из предыдущих данных, то выбросы могут испортить нам общую картину

In [533]:
df[(df['is_pay']==1)&(df['date']<='2023-04-31')].describe()

,timestamp,sum_payment,id_product,is_pay
count,6.642000e+03,6642.000000,6641.000000,6642.0
mean,1.682224e+09,21.251129,11.440897,1.0
std,5.790047e+05,15.382642,5.723223,0.0
min,1.680331e+09,10.000000,0.000000,1.0
25%,1.681852e+09,13.000000,7.000000,1.0
50%,1.682384e+09,19.000000,11.000000,1.0
75%,1.682710e+09,28.000000,16.000000,1.0
max,1.682888e+09,1000.000000,21.000000,1.0


### Посмотреть, есть ли выбросы в данных.

Есть клиенты, которые сделали крупную покупку сразу на 1000, тогда как среднее значение чека даже с учетом выброса - 21. Чтобы скорректировать влияние выбросов на метрики, попробуем удалить выброс, у нас много измерений, в йелом удаление не должно оказать отрицательного влияния

In [538]:
df[(df['is_pay']==1)&(df['sum_payment']==1000)&(df['date']<='2023-04-31')]

,timestamp,id_user,sum_payment,group,city,id_product,is_pay,date
2705,1681354189,user_0,1000,NaN,Москва,NaN,1,2023-04-13


In [ ]:
df = df.drop(labels=2705)

df = df.reset_index()

In [540]:
# df[(df['is_pay']==1)&(df['sum_payment']==1000)&(df['date']<='2023-04-31')]

In [553]:
round(df[(df['is_pay']==1)&(df['date']<='2023-04-31')].sum_payment.mean(), 2)

21.1

Добавим поля, которые будут обозначать дату первой покупки и последний день, спустя неделю после покупки.

### Построить методику расчета целевой метрики.

In [554]:
def get_dates(df):
    first_payment = df[(df['is_pay']==1)].groupby('id_user')['date'].min()
    last_date = pd.to_datetime(first_payment) + pd.to_timedelta(7, unit='d')
    result = pd.DataFrame(data=zip(list(first_payment.index), first_payment, last_date), columns=['id_user', 'first_payment', 'last_date'])
    return result

result = get_dates(df[(df['date']<='2023-04-31')])
result['id_user'] = result['id_user'].astype(object)
df['id_user'] = df['id_user'].astype(object)
df_before = df.merge(result, on='id_user', how='left')[(df['date']<='2023-04-31')]
df_before.head(3)

,index,timestamp,id_user,sum_payment,group,city,id_product,is_pay,date,first_payment,last_date
0,0,1680330573,user_9903,27,NaN,Санкт-Петербург,4.0,1,2023-04-01,2023-04-01,2023-04-08
1,1,1680332652,user_6732,0,NaN,Рязань,1.0,0,2023-04-01,2023-04-04,2023-04-11
2,2,1680378039,user_4199,0,NaN,Москва,3.0,0,2023-04-01,2023-04-15,2023-04-22


In [564]:
payments_before = df_before[(df_before['date']>=df_before['first_payment'])&(df_before['date']<=df_before['last_date'])]
mean_payments_before = payments_before.groupby('id_user')['sum_payment'].mean()

m1 = mean_payments_before.mean()
sigma_1 = mean_payments_before.std()
delta_effect = m1 * 0.1
print('Среднее значение метрики до АБ эксперимента:', m1)

Среднее значение метрики до АБ эксперимента: 15.201310749186485


Оценим длительность эксперимента 

In [565]:
delta_effect, sigma_1

(1.5201310749186485, 9.365654969171416)

In [566]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=0.05, beta=0.2):
    z = sps.norm.ppf(1 - alpha/2) + sps.norm.ppf(1-beta)
    n = (k+1) * z ** 2 * (sigma_1 ** 2 + sigma_2 **2 / k) / (delta_effect ** 2)
    return n

In [567]:
k = 1
alpha = 0.05
beta = 0.2
d = int(duration(k, delta_effect, sigma_1, sigma_1, alpha, beta)) + 1
print('Для наблюдения эффекта в 10% с вероятность 95% необходима выборка из', d, 'человек, которые бы совершали покупки, по ', d//2, 'человек в каждой группе')

Для наблюдения эффекта в 10% с вероятность 95% необходима выборка из 1192 человек, которые бы совершали покупки, по  596 человек в каждой группе


In [568]:
users_before = payments_before.id_user.nunique()
users_before

4731

Для эксперимента нам нужно 1192 пользователя, при том, что всего уникальных юзеров за месяц 4731. Кажется, что длительность эксперимента в месяц полностью покрывает наши потребности, можем оставить длительность такой, какую наметили изначально. Приступаем к оценке эксперимента

### Рассчитать целевую метрику на день окончания теста (рассчитанной в п1) для группы A и B, рассчитать эффект, p_value. Посмотреть, есть ли выбросы в данных.

Посмотрим на данные после начала эксперимента

In [569]:
df[(df['is_pay']==1)&(df['date']>'2023-04-31')].describe()

,index,timestamp,sum_payment,id_product,is_pay
count,9142.000000,9.142000e+03,9142.000000,9140.000000,9142.0
mean,41087.971013,1.684902e+09,21.302013,11.453939,1.0
std,8834.267316,5.914897e+05,17.480641,5.647674,0.0
min,25764.000000,1.682921e+09,10.000000,0.000000,1.0
25%,33442.000000,1.684554e+09,13.000000,7.000000,1.0
50%,41057.000000,1.685069e+09,19.000000,11.000000,1.0
75%,48784.000000,1.685395e+09,28.000000,16.000000,1.0
max,56402.000000,1.685567e+09,1000.000000,21.000000,1.0


Ситуация с выбросами точно такая же - присутствуют большие значения платежей, которые влияют на метрику среднего. Скорректируем эти точечные выбросы значением среднего

In [482]:
df[(df['is_pay']==1)&(df['sum_payment']==1000)&(df['date']>'2023-04-31')]

,timestamp,id_user,sum_payment,group,city,id_product,is_pay,date
26242,1683320400,user_11998,1000.0,A,Москва,NaN,1,2023-05-06
26272,1683320400,user_11996,1000.0,A,Москва,NaN,1,2023-05-06


In [571]:
df = df.drop(labels=26272)
df = df.drop(labels=26242)

df = df.reset_index()

In [572]:
result_AB = get_dates(df[(df['date']>'2023-04-31')])
result_AB['id_user'] = result_AB['id_user'].astype(object)
df['id_user'] = df['id_user'].astype(object)
df_AB = df.merge(result_AB, on='id_user', how='left')[(df['date']>'2023-04-31')]
df_AB.head(3)

,level_0,index,timestamp,id_user,sum_payment,group,city,id_product,is_pay,date,first_payment,last_date
25758,25758,25759,1682972952,user_8276,0,A,Москва,10.0,0,2023-05-01,2023-05-18,2023-05-25
25759,25759,25760,1682964072,user_4627,0,B,Рязань,18.0,0,2023-05-01,2023-05-26,2023-06-02
25760,25760,25761,1682894626,user_7623,0,B,Санкт-Петербург,21.0,0,2023-05-01,2023-05-13,2023-05-20


Посчитаем метрики эксперимента по обеим группам. Сформируем группы, посчитаем средний доход от одного пользователя по А и Б

In [573]:
group_a = df_AB[df_AB['group'] == 'A']
group_b = df_AB[df_AB['group'] == 'B']

In [574]:
payments_A = group_a[(group_a['date']>=group_a['first_payment'])&(group_a['date']<=group_a['last_date'])]
mean_payments_A = payments_A.groupby('id_user')['sum_payment'].mean()
std_payment_A = payments_A.groupby('id_user')['sum_payment'].std()

payments_B = group_b[(group_b['date']>=group_b['first_payment'])&(group_b['date']<=group_b['last_date'])]
mean_payments_B = payments_B.groupby('id_user')['sum_payment'].mean()
std_payment_B = payments_B.groupby('id_user')['sum_payment'].std()

print('Cредний доход от пользователя в группе А:', mean_payments_A.mean())
print('Cредний доход от пользователя в группе B:', mean_payments_B.mean())

Cредний доход от пользователя в группе А: 15.954459013689753
Cредний доход от пользователя в группе B: 15.671286688334279


Посмотрим, набрали ли мы нужное количество измерений (платежей), чтобы их сравнивать 

In [575]:
alpha = 0.05
p = sps.ttest_ind(mean_payments_A, mean_payments_B).pvalue
if p < alpha: 
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергнуть нулевую гипотезу')

Нет оснований отвергнуть нулевую гипотезу


У нас получилось, что средние значения платежа по группам А, B очень похожи. Нельзя отвергнуть нулевую гипотезу о равенстве средних.